### Import Libraries and Data

In [183]:
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

In [184]:
# Rental History Data
one_bed_flat = pd.read_csv('../data/raw/rental_history/one_bed_flat.csv')
two_bed_flat = pd.read_csv('../data/raw/rental_history/two_bed_flat.csv')
three_bed_flat = pd.read_csv('../data/raw/rental_history/three_bed_flat.csv')
two_bed_house = pd.read_csv('../data/raw/rental_history/two_bed_house.csv')
three_bed_house = pd.read_csv('../data/raw/rental_history/three_bed_house.csv')
four_bed_house = pd.read_csv('../data/raw/rental_history/four_bed_house.csv')
all_properties = pd.read_csv('../data/raw/rental_history/all_properties.csv')

# Domain Rental Data
domain_one_bed_flat = pd.read_csv('../data/curated/domain_one_bed_flat_rent.csv')
domain_two_bed_flat = pd.read_csv('../data/curated/domain_two_bed_flat_rent.csv')
domain_three_bed_flat = pd.read_csv('../data/curated/domain_three_bed_flat_rent.csv')
domain_two_bed_house = pd.read_csv('../data/curated/domain_two_bed_house_rent.csv')
domain_three_bed_house = pd.read_csv('../data/curated/domain_three_bed_house_rent.csv')
domain_four_bed_house = pd.read_csv('../data/curated/domain_four_bed_house.csv')
domain_all_properties = pd.read_csv('../data/curated/domain_all_properties_rent.csv')

# Other engineered feature sets 
crimes = pd.read_csv('../data/curated/crimes.csv')
population = pd.read_csv('../data/curated/final_population.csv')
education = pd.read_csv('../data/curated/education_df.csv')
urban_landmarks = pd.read_csv('../data/raw/urban_landmarks_features.csv')
pt_distances = pd.read_csv('../data/curated/suburb_transport_distances.csv')
income = pd.read_csv('../data/curated/income.csv')

### Formatting Rental Dataframes

In [185]:
def clean_domain_df(df):
    """
    This function cleans the domain dataframes by removing
    the 'Unnamed:' column, renaming median_rent to 'sep_median'
    (for a standardised column name as in rental history dfs) and
    also creates a year column and inputs the relevant year that
    the data is from - 2024. 
    """

    # Drop columns that contain 'Unnamed:' in their name
    df = df.loc[:, ~df.columns.str.contains('^Unnamed:')]
    
    # Rename the 'median_rent' column to 'sep_median'
    if 'median_rent' in df.columns:
        df = df.rename(columns={'median_rent': 'sep_median'})
    
    # Add a 'year' column with value 2024 for each row
    df['year'] = 2024

    # Reorder columns to make 'year' the second column
    cols = list(df.columns)
    cols.insert(1, cols.pop(cols.index('year')))
    df = df[cols]
    
    return df

# Apply the clean_domain_df function to all the domain dataframes
domain_one_bed_flat = clean_domain_df(domain_one_bed_flat)
domain_two_bed_flat = clean_domain_df(domain_two_bed_flat)
domain_three_bed_flat = clean_domain_df(domain_three_bed_flat)
domain_two_bed_house = clean_domain_df(domain_two_bed_house)
domain_three_bed_house = clean_domain_df(domain_three_bed_house)
domain_four_bed_house = clean_domain_df(domain_four_bed_house)
domain_all_properties = clean_domain_df(domain_all_properties)


Impute the Sep median price from scraped properties into the rental history dataframes

In [186]:
def impute_sep_2024_rental_data(rental_history_df, domain_df):
    """
    This function retrieves all the median rental prices in 
    September from the domain dataframes and then imputes
    them into the rental history dataframes where the year
    is 2024 and month is September. 
    """

    # Merge rental_history_df with domain_df on 'suburb' to keep all years from rental_history_df
    merged_df = pd.merge(rental_history_df, domain_df[['suburb', 'year', 'sep_median']],
                         on=['suburb'], how='left', suffixes=('', '_domain'))
    
    # Replace sep_median values with domain values only for rows where year == 2024
    condition = (merged_df['year'] == 2024) & merged_df['sep_median_domain'].notna()
    merged_df.loc[condition, 'sep_median'] = merged_df.loc[condition, 'sep_median_domain']
    
    # Drop the domain-specific columns used for imputation
    merged_df.drop(columns=['sep_median_domain', 'year_domain'], inplace=True)

    # Filter the dataframe to keep only the suburbs that appear 9 or more times
    suburb_counts = merged_df['suburb'].value_counts()
    suburbs_to_keep = suburb_counts[suburb_counts >= 9].index
    merged_df = merged_df[merged_df['suburb'].isin(suburbs_to_keep)]
    
    # Drop the sep_median column from the domain DataFrame
    domain_df = domain_df.drop(columns=['year', 'sep_median', 'num_properties'], errors='ignore')
    
    return merged_df, domain_df

# Apply the function to each dataset 
one_bed_flat, domain_one_bed_flat = impute_sep_2024_rental_data(one_bed_flat, domain_one_bed_flat)
two_bed_flat, domain_two_bed_flat = impute_sep_2024_rental_data(two_bed_flat, domain_two_bed_flat)
three_bed_flat, domain_three_bed_flat = impute_sep_2024_rental_data(three_bed_flat, domain_three_bed_flat)
two_bed_house, domain_two_bed_house = impute_sep_2024_rental_data(two_bed_house, domain_two_bed_house)
three_bed_house, domain_three_bed_house = impute_sep_2024_rental_data(three_bed_house, domain_three_bed_house)
four_bed_house, domain_four_bed_house = impute_sep_2024_rental_data(four_bed_house, domain_four_bed_house)
all_properties, domain_all_properties = impute_sep_2024_rental_data(all_properties, domain_all_properties)

### Combining All Feature Sets

In [187]:
# Merge one_bed_flat
one_bed_flat_merged = pd.merge(one_bed_flat, crimes, on=['suburb', 'year'], how='outer')
one_bed_flat_merged = pd.merge(one_bed_flat_merged, income, on=['suburb', 'year'], how='outer')
one_bed_flat_merged = pd.merge(one_bed_flat_merged, population, left_on=['suburb', 'year'], right_on=['sa2_name', 'year'], how='outer')
one_bed_flat_merged = pd.merge(one_bed_flat_merged, education, on='suburb', how='inner')
one_bed_flat_merged = pd.merge(one_bed_flat_merged, urban_landmarks, left_on='suburb', right_on='gazetted_locality', how='inner')
one_bed_flat_merged = pd.merge(one_bed_flat_merged, pt_distances, on='suburb', how='inner')
one_bed_flat_merged = pd.merge(one_bed_flat_merged, domain_one_bed_flat, on='suburb', how='inner')

# Filter rows to retain only matching suburbs in one_bed_flat and the population dataframe
one_bed_flat_merged = one_bed_flat_merged[one_bed_flat_merged['suburb'].isin(population['sa2_name'])]


# Merge two_bed_flat
two_bed_flat_merged = pd.merge(two_bed_flat, crimes, on=['suburb', 'year'], how='outer')
two_bed_flat_merged = pd.merge(two_bed_flat_merged, income, on=['suburb', 'year'], how='outer')
two_bed_flat_merged = pd.merge(two_bed_flat_merged, population, left_on=['suburb', 'year'], right_on=['sa2_name', 'year'], how='outer')
two_bed_flat_merged = pd.merge(two_bed_flat_merged, education, on='suburb', how='inner')
two_bed_flat_merged = pd.merge(two_bed_flat_merged, urban_landmarks, left_on='suburb', right_on='gazetted_locality', how='inner')
two_bed_flat_merged = pd.merge(two_bed_flat_merged, pt_distances, on='suburb', how='inner')
two_bed_flat_merged = pd.merge(two_bed_flat_merged, domain_two_bed_flat, on='suburb', how='inner')

# Filter rows to retain only matching suburbs in two_bed_flat and the population dataframe
two_bed_flat_merged = two_bed_flat_merged[two_bed_flat_merged['suburb'].isin(population['sa2_name'])]


# Merge three_bed_flat
three_bed_flat_merged = pd.merge(three_bed_flat, crimes, on=['suburb', 'year'], how='outer')
three_bed_flat_merged = pd.merge(three_bed_flat_merged, income, on=['suburb', 'year'], how='outer')
three_bed_flat_merged = pd.merge(three_bed_flat_merged, population, left_on=['suburb', 'year'], right_on=['sa2_name', 'year'], how='outer')
three_bed_flat_merged = pd.merge(three_bed_flat_merged, education, on='suburb', how='inner')
three_bed_flat_merged = pd.merge(three_bed_flat_merged, urban_landmarks, left_on='suburb', right_on='gazetted_locality', how='inner')
three_bed_flat_merged = pd.merge(three_bed_flat_merged, pt_distances, on='suburb', how='inner')
three_bed_flat_merged = pd.merge(three_bed_flat_merged, domain_three_bed_flat, on='suburb', how='inner')

# Filter rows to retain only matching suburbs in three_bed_flat and the population dataframe
three_bed_flat_merged = three_bed_flat_merged[three_bed_flat_merged['suburb'].isin(population['sa2_name'])]


# Merge two_bed_house
two_bed_house_merged = pd.merge(two_bed_house, crimes, on=['suburb', 'year'], how='outer')
two_bed_house_merged = pd.merge(two_bed_house_merged, income, on=['suburb', 'year'], how='outer')
two_bed_house_merged = pd.merge(two_bed_house_merged, population, left_on=['suburb', 'year'], right_on=['sa2_name', 'year'], how='outer')
two_bed_house_merged = pd.merge(two_bed_house_merged, education, on='suburb', how='inner')
two_bed_house_merged = pd.merge(two_bed_house_merged, urban_landmarks, left_on='suburb', right_on='gazetted_locality', how='inner')
two_bed_house_merged = pd.merge(two_bed_house_merged, pt_distances, on='suburb', how='inner')
two_bed_house_merged = pd.merge(two_bed_house_merged, domain_two_bed_house, on='suburb', how='inner')

# Filter rows to retain only matching suburbs in two_bed_house and the population dataframe
two_bed_house_merged = two_bed_house_merged[two_bed_house_merged['suburb'].isin(population['sa2_name'])]


# Merge three_bed_house
three_bed_house_merged = pd.merge(three_bed_house, crimes, on=['suburb', 'year'], how='outer')
three_bed_house_merged = pd.merge(three_bed_house_merged, income, on=['suburb', 'year'], how='outer')
three_bed_house_merged = pd.merge(three_bed_house_merged, population, left_on=['suburb', 'year'], right_on=['sa2_name', 'year'], how='outer')
three_bed_house_merged = pd.merge(three_bed_house_merged, education, on='suburb', how='inner')
three_bed_house_merged = pd.merge(three_bed_house_merged, urban_landmarks, left_on='suburb', right_on='gazetted_locality', how='inner')
three_bed_house_merged = pd.merge(three_bed_house_merged, pt_distances, on='suburb', how='inner')
three_bed_house_merged = pd.merge(three_bed_house_merged, domain_three_bed_house, on='suburb', how='inner')

# Filter rows to retain only matching suburbs in three_bed_house and the population dataframe
three_bed_house_merged = three_bed_house_merged[three_bed_house_merged['suburb'].isin(population['sa2_name'])]


# Merge four_bed_house
four_bed_house_merged = pd.merge(four_bed_house, crimes, on=['suburb', 'year'], how='outer')
four_bed_house_merged = pd.merge(four_bed_house_merged, income, on=['suburb', 'year'], how='outer')
four_bed_house_merged = pd.merge(four_bed_house_merged, population, left_on=['suburb', 'year'], right_on=['sa2_name', 'year'], how='outer')
four_bed_house_merged = pd.merge(four_bed_house_merged, education, on='suburb', how='inner')
four_bed_house_merged = pd.merge(four_bed_house_merged, urban_landmarks, left_on='suburb', right_on='gazetted_locality', how='inner')
four_bed_house_merged = pd.merge(four_bed_house_merged, pt_distances, on='suburb', how='inner')
four_bed_house_merged = pd.merge(four_bed_house_merged, domain_four_bed_house, on='suburb', how='inner')

# Filter rows to retain only matching suburbs in four_bed_house and the population dataframe
four_bed_house_merged = four_bed_house_merged[four_bed_house_merged['suburb'].isin(population['sa2_name'])]


# Merge all_properties
all_properties_merged = pd.merge(all_properties, crimes, on=['suburb', 'year'], how='outer')
all_properties_merged = pd.merge(all_properties_merged, income, on=['suburb', 'year'], how='outer')
all_properties_merged = pd.merge(all_properties_merged, population, left_on=['suburb', 'year'], right_on=['sa2_name', 'year'], how='outer')
all_properties_merged = pd.merge(all_properties_merged, education, on='suburb', how='inner')
all_properties_merged = pd.merge(all_properties_merged, urban_landmarks, left_on='suburb', right_on='gazetted_locality', how='inner')
all_properties_merged = pd.merge(all_properties_merged, pt_distances, on='suburb', how='inner')
all_properties_merged = pd.merge(all_properties_merged, domain_all_properties, on='suburb', how='inner')

# Filter rows to retain only matching suburbs in all_properties and the population dataframe
all_properties_merged = all_properties_merged[all_properties_merged['suburb'].isin(population['sa2_name'])]


one_bed_flat_merged

,suburb,year,dec_median,jun_median,mar_median,sep_median,offence_division_1,offence_division_2,offence_division_3,offence_division_4,...,distance_to_restaurant,distance_to_supermarket,nearest_transport_avg_distance,distance_to_cbd,median_bath,median_parkings,furnished_count,unfurnished_count,pets_allowed,pets_not_allowed
0,abbotsford,2016,380.0,380.0,380.0,380.0,107.0,1065.0,76.0,59.0,...,1.141601,1.083238,1.110000,5.55,1,0,2,10,4,8
1,abbotsford,2017,400.0,390.0,390.0,395.0,138.0,1019.0,64.0,69.0,...,1.141601,1.083238,1.110000,5.55,1,0,2,10,4,8
2,abbotsford,2018,410.0,400.0,400.0,400.0,100.0,1162.0,88.0,84.0,...,1.141601,1.083238,1.110000,5.55,1,0,2,10,4,8
3,abbotsford,2019,420.0,420.0,410.0,420.0,175.0,1053.0,178.0,114.0,...,1.141601,1.083238,1.110000,5.55,1,0,2,10,4,8
4,abbotsford,2020,390.0,418.0,420.0,410.0,145.0,985.0,151.0,89.0,...,1.141601,1.083238,1.110000,5.55,1,0,2,10,4,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1615,yarraville,2023,295.0,270.0,275.0,280.0,96.0,633.0,68.0,31.0,...,1.032864,0.937960,1.826667,10.21,1,1,0,1,0,1
1616,yarraville,2024,325.0,325.0,325.0,320.0,101.0,607.0,60.0,40.0,...,1.032864,0.937960,1.826667,10.21,1,1,0,1,0,1
1617,yarraville,2025,NaN,NaN,NaN,NaN,102.0,604.0,63.0,40.0,...,1.032864,0.937960,1.826667,10.21,1,1,0,1,0,1
1618,yarraville,2026,NaN,NaN,NaN,NaN,103.0,600.0,65.0,39.0,...,1.032864,0.937960,1.826667,10.21,1,1,0,1,0,1


In [188]:
# Drop all other suburb column names. Only keep the first suburb column 
def clean_suburb_cols(df):
    """
    This function removes all duplicated of the suburb column name 
    from the merged dataframes. The duplicate suburb column name 
    could be 'Unnamed', 'sa2_name' or 'gazetted_locality'.
    """
    df = df.loc[:, ~df.columns.str.contains('Unnamed')]  # removes the duplicate 'suburb' column
    columns_to_drop = ['sa2_name', 'gazetted_locality']
    df = df.drop(columns=[col for col in columns_to_drop if col in df.columns])

    return df

# Clean the column names
one_bed_flat_merged = clean_suburb_cols(one_bed_flat_merged)
two_bed_flat_merged = clean_suburb_cols(two_bed_flat_merged)
three_bed_flat_merged = clean_suburb_cols(three_bed_flat_merged)
two_bed_house_merged = clean_suburb_cols(two_bed_house_merged)
three_bed_house_merged = clean_suburb_cols(three_bed_house_merged)
four_bed_house_merged = clean_suburb_cols(four_bed_house_merged)
all_properties_merged = clean_suburb_cols(all_properties_merged)

#----- DO THIS FOR ALLLLLLLLLLL
# Define the columns to check for NaN values
median_columns = ['dec_median', 'jun_median', 'mar_median', 'sep_median']

# Drop rows where year is less than 2025 and NaN values exist in any of the median columns
one_bed_flat_merged = one_bed_flat_merged[~((one_bed_flat_merged['year'] < 2025) & (one_bed_flat_merged[median_columns].isnull().any(axis=1)))]

# Display the cleaned dataframe
print(one_bed_flat_merged)



          suburb  year  dec_median  jun_median  mar_median  sep_median  \
0     abbotsford  2016       380.0       380.0       380.0       380.0   
1     abbotsford  2017       400.0       390.0       390.0       395.0   
2     abbotsford  2018       410.0       400.0       400.0       400.0   
3     abbotsford  2019       420.0       420.0       410.0       420.0   
4     abbotsford  2020       390.0       418.0       420.0       410.0   
...          ...   ...         ...         ...         ...         ...   
1615  yarraville  2023       295.0       270.0       275.0       280.0   
1616  yarraville  2024       325.0       325.0       325.0       320.0   
1617  yarraville  2025         NaN         NaN         NaN         NaN   
1618  yarraville  2026         NaN         NaN         NaN         NaN   
1619  yarraville  2027         NaN         NaN         NaN         NaN   

      offence_division_1  offence_division_2  offence_division_3  \
0                  107.0              1065.

### Save All Properties Dataframe for Visualisation Purposes

In [189]:
# Save df to a CSV file
all_properties_merged.to_csv('../data/curated/all_properties_merged.csv', index=False)

### Create Train & Test Sets

In [193]:
def train_val_test_sets(df):
    """
    This function first splits the dataframe into training, validation, 
    and testing sets based on the 'year' column:
    - Training set includes data from the years 2016-2021.
    - Validation set includes data from the years 2022-2024.
    - Testing set includes data from the years 2025-2027.

    It then merges additional columns that are not part of the 
    features specific to years or target columns back with the 
    respective sets based on matching suburbs.

    The function then returns the follow dataframes:
    - X_train: Training feature set.
    - X_val: Validation feature set.
    - X_test: Testing feature set.
    - y_train: Training target set.
    - y_val: Validation target set.
    - y_test: Testing target set.
    """
    # Define the year ranges for training, validation, and testing sets
    train_years = range(2016, 2022)
    val_years = range(2022, 2025)
    test_years = range(2025, 2028)

    # Columns to include in X (specific to the years) and y splits
    feature_year_cols = ['suburb', 'year', 'offence_division_1', 'offence_division_2', 'offence_division_3', 'offence_division_4', 'offence_division_5', 'offence_division_6', 'erp']
    target_columns = ['dec_median', 'jun_median', 'mar_median', 'sep_median']

    # Add the income columns that start with 'hi_' and end with '_tot' using regex
    regex_pattern = r'^hi_.*_tot$'
    hi_tot_cols = df.filter(regex=regex_pattern).columns.tolist()
    # Combine to the feature_year_cols
    feature_year_cols += hi_tot_cols

    # Split features (specific to the years) and target data
    X = df[feature_year_cols]
    y = df[target_columns]

    # Split the dataframe into training, validation, and testing sets based on the year
    X_train = X[X['year'].isin(train_years)]
    X_val = X[X['year'].isin(val_years)]
    X_test = X[X['year'].isin(test_years)]

    # Align target sets with the corresponding feature sets
    y_train = y.loc[X_train.index]
    y_val = y.loc[X_val.index]
    y_test = y.loc[X_test.index]

    # Extract other columns not in feature_year_cols or target_columns, including 'suburb'
    other_columns = df.drop(columns=feature_year_cols + target_columns).columns
    other_data = df[other_columns].copy()
    other_data['suburb'] = df['suburb']  # Ensure 'suburb' is included

    # Merge the 'other' data back with the matching suburbs, irrespective of the year
    X_train = X_train.merge(other_data, on='suburb', how='left')
    X_val = X_val.merge(other_data, on='suburb', how='left')
    X_test = X_test.merge(other_data, on='suburb', how='left')

    # Drop 'suburb' and 'year' from the feature sets
    #X_train = X_train.drop(columns=['suburb', 'year'])
    X_val = X_val.drop(columns=['suburb', 'year'])
    X_test = X_test.drop(columns=['suburb', 'year'])

    print(y_train.isnull().sum())

    # # Remove rows with NaN values in y_train and X_train
    # valid_train_indices = y_train.dropna().index
    # X_train = X_train.loc[valid_train_indices]
    # y_train = y_train.loc[valid_train_indices]

    # # Remove rows with NaN values in y_val and X_val
    # valid_val_indices = y_val.dropna().index
    # X_val = X_val.loc[valid_val_indices]
    # y_val = y_val.loc[valid_val_indices]

    return X_train, X_val, X_test, y_train, y_val, y_test

# Create training, validation, and test sets for each property type
X_train_one_bed, X_val_one_bed, X_test_one_bed, y_train_one_bed, y_val_one_bed, y_test_one_bed = train_val_test_sets(one_bed_flat_merged)
X_train_two_bed, X_val_two_bed, X_test_two_bed, y_train_two_bed, y_val_two_bed, y_test_two_bed = train_val_test_sets(two_bed_flat_merged)
X_train_three_bed, X_val_three_bed, X_test_three_bed, y_train_three_bed, y_val_three_bed, y_test_three_bed = train_val_test_sets(three_bed_flat_merged)
X_train_two_bed_house, X_val_two_bed_house, X_test_two_bed_house, y_train_two_bed_house, y_val_two_bed_house, y_test_two_bed_house = train_val_test_sets(two_bed_house_merged)
X_train_three_bed_house, X_val_three_bed_house, X_test_three_bed_house, y_train_three_bed_house, y_val_three_bed_house, y_test_three_bed_house = train_val_test_sets(three_bed_house_merged)
X_train_four_bed_house, X_val_four_bed_house, X_test_four_bed_house, y_train_four_bed_house, y_val_four_bed_house, y_test_four_bed_house = train_val_test_sets(four_bed_house_merged)
X_train_all_properties, X_val_all_properties, X_test_all_properties, y_train_all_properties, y_val_all_properties, y_test_all_properties = train_val_test_sets(all_properties_merged)

X_train_two_bed.drop_duplicates()

dec_median    0
jun_median    0
mar_median    0
sep_median    0
dtype: int64
dec_median    0
jun_median    0
mar_median    0
sep_median    0
dtype: int64
dec_median    12
jun_median    12
mar_median    12
sep_median    12
dtype: int64
dec_median    36
jun_median    36
mar_median    36
sep_median    36
dtype: int64
dec_median    12
jun_median    12
mar_median    12
sep_median    12
dtype: int64
dec_median    6
jun_median    6
mar_median    6
sep_median    6
dtype: int64
dec_median    0
jun_median    0
mar_median    0
sep_median    0
dtype: int64


,suburb,year,offence_division_1,offence_division_2,offence_division_3,offence_division_4,offence_division_5,offence_division_6,erp,hi_1_149_tot,...,distance_to_restaurant,distance_to_supermarket,nearest_transport_avg_distance,distance_to_cbd,median_bath,median_parkings,furnished_count,unfurnished_count,pets_allowed,pets_not_allowed
0,abbotsford,2016,107.0,1065.0,76.0,59.0,49.0,0.0,8770.0,27.0,...,1.141601,1.083238,1.110000,5.55,2,1,3,29,7,25
12,abbotsford,2017,138.0,1019.0,64.0,69.0,126.0,0.0,9291.0,27.0,...,1.141601,1.083238,1.110000,5.55,2,1,3,29,7,25
24,abbotsford,2018,100.0,1162.0,88.0,84.0,114.0,2.0,9527.0,27.0,...,1.141601,1.083238,1.110000,5.55,2,1,3,29,7,25
36,abbotsford,2019,175.0,1053.0,178.0,114.0,69.0,3.0,9594.0,27.0,...,1.141601,1.083238,1.110000,5.55,2,1,3,29,7,25
48,abbotsford,2020,145.0,985.0,151.0,89.0,73.0,129.0,9672.0,27.0,...,1.141601,1.083238,1.110000,5.55,2,1,3,29,7,25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12180,yarraville,2017,100.0,601.0,42.0,38.0,55.0,5.0,15804.0,37.0,...,1.032864,0.937960,1.826667,10.21,1,1,0,9,2,7
12192,yarraville,2018,67.0,662.0,43.0,56.0,43.0,5.0,15991.0,38.0,...,1.032864,0.937960,1.826667,10.21,1,1,0,9,2,7
12204,yarraville,2019,127.0,596.0,41.0,40.0,48.0,2.0,16092.0,38.0,...,1.032864,0.937960,1.826667,10.21,1,1,0,9,2,7
12216,yarraville,2020,82.0,643.0,38.0,52.0,61.0,28.0,16068.0,39.0,...,1.032864,0.937960,1.826667,10.21,1,1,0,9,2,7


Check to see all the X columns are the same 

In [124]:
def compare_feature_columns(train, val, test):
    """
    This function compares columns of the training, validation, and 
    testing feature dataframes (X). A dictionary is then returned
    indicating if any columns are missing in each set or if all
    the colums are the same. 
    """
    comparison_result = {}
    # Check if columns match between train, validation, and test sets
    train_val_match = train.columns.equals(val.columns)
    train_test_match = train.columns.equals(test.columns)
    val_test_match = val.columns.equals(test.columns)
    
    if not (train_val_match and train_test_match and val_test_match):
        missing_in_val = set(train.columns) - set(val.columns)
        missing_in_train_val = set(val.columns) - set(train.columns)
        missing_in_test = set(train.columns) - set(test.columns)
        missing_in_train_test = set(test.columns) - set(train.columns)
        missing_in_val_test = set(val.columns) - set(test.columns)
        missing_in_test_val = set(test.columns) - set(val.columns)

        comparison_result = {
            "Columns missing in validation set compared to train": list(missing_in_val),
            "Columns missing in train set compared to validation": list(missing_in_train_val),
            "Columns missing in test set compared to train": list(missing_in_test),
            "Columns missing in train set compared to test": list(missing_in_train_test),
            "Columns missing in test set compared to validation": list(missing_in_val_test),
            "Columns missing in validation set compared to test": list(missing_in_test_val),
        }
    else:
        comparison_result = "Columns are the same in all three sets (train, validation, test)."

    return comparison_result

# List of training, validation, and testing DataFrames to compare
feature_dfs = {
    "One Bed": (X_train_one_bed, X_val_one_bed, X_test_one_bed),
    "Two Bed": (X_train_two_bed, X_val_two_bed, X_test_two_bed),
    "Three Bed": (X_train_three_bed, X_val_three_bed, X_test_three_bed),
    "Two Bed House": (X_train_two_bed_house, X_val_two_bed_house, X_test_two_bed_house),
    "Three Bed House": (X_train_three_bed_house, X_val_three_bed_house, X_test_three_bed_house),
    "Four Bed House": (X_train_four_bed_house, X_val_four_bed_house, X_test_four_bed_house),
    "All Properties": (X_train_all_properties, X_val_all_properties, X_test_all_properties)
}

# Compare columns for each triplet of training, validation, and testing sets
comparison_results = {name: compare_feature_columns(train, val, test) for name, (train, val, test) in feature_dfs.items()}

comparison_results 

NameError: name 'X_train_two_bed' is not defined

### Check Null Values

In [ ]:
# List of dataframes to check for missing values
dataframes = {
    'X_train_one_bed': X_train_one_bed,
    'X_val_one_bed': X_val_one_bed,
    'X_test_one_bed': X_test_one_bed,
    'y_train_one_bed': y_train_one_bed,
    'y_val_one_bed': y_val_one_bed,
    'y_test_one_bed': y_test_one_bed,
    
    'X_train_two_bed': X_train_two_bed,
    'X_val_two_bed': X_val_two_bed,
    'X_test_two_bed': X_test_two_bed,
    'y_train_two_bed': y_train_two_bed,
    'y_val_two_bed': y_val_two_bed,
    'y_test_two_bed': y_test_two_bed,
    
    'X_train_three_bed': X_train_three_bed,
    'X_val_three_bed': X_val_three_bed,
    'X_test_three_bed': X_test_three_bed,
    'y_train_three_bed': y_train_three_bed,
    'y_val_three_bed': y_val_three_bed,
    'y_test_three_bed': y_test_three_bed,
    
    'X_train_two_bed_house': X_train_two_bed_house,
    'X_val_two_bed_house': X_val_two_bed_house,
    'X_test_two_bed_house': X_test_two_bed_house,
    'y_train_two_bed_house': y_train_two_bed_house,
    'y_val_two_bed_house': y_val_two_bed_house,
    'y_test_two_bed_house': y_test_two_bed_house,
    
    'X_train_three_bed_house': X_train_three_bed_house,
    'X_val_three_bed_house': X_val_three_bed_house,
    'X_test_three_bed_house': X_test_three_bed_house,
    'y_train_three_bed_house': y_train_three_bed_house,
    'y_val_three_bed_house': y_val_three_bed_house,
    'y_test_three_bed_house': y_test_three_bed_house,
    
    'X_train_four_bed_house': X_train_four_bed_house,
    'X_val_four_bed_house': X_val_four_bed_house,
    'X_test_four_bed_house': X_test_four_bed_house,
    'y_train_four_bed_house': y_train_four_bed_house,
    'y_val_four_bed_house': y_val_four_bed_house,
    'y_test_four_bed_house': y_test_four_bed_house,
    
    'X_train_all_properties': X_train_all_properties,
    'X_val_all_properties': X_val_all_properties,
    'X_test_all_properties': X_test_all_properties,
    'y_train_all_properties': y_train_all_properties,
    'y_val_all_properties': y_val_all_properties,
    'y_test_all_properties': y_test_all_properties,
}

In [ ]:
# Collecting rows with missing values for each dataframe
missing_rows_summary = {}

for name, df in dataframes.items():
    rows_with_missing = df[df.isnull().any(axis=1)]
    if not rows_with_missing.empty:
        missing_rows_summary[name] = rows_with_missing

print(missing_rows_summary)

# Check for missing values in each dataframe by columns 
missing_values_summary = {}
for name, df in dataframes.items():
    missing_values = df.isnull().sum()
    columns_with_missing = missing_values[missing_values > 0]
    if not columns_with_missing.empty:
        missing_values_summary[name] = columns_with_missing

print(missing_values_summary)

{'y_test_one_bed':       dec_median  jun_median  mar_median  sep_median
54           NaN         NaN         NaN         NaN
55           NaN         NaN         NaN         NaN
56           NaN         NaN         NaN         NaN
57           NaN         NaN         NaN         NaN
58           NaN         NaN         NaN         NaN
...          ...         ...         ...         ...
9715         NaN         NaN         NaN         NaN
9716         NaN         NaN         NaN         NaN
9717         NaN         NaN         NaN         NaN
9718         NaN         NaN         NaN         NaN
9719         NaN         NaN         NaN         NaN

[2376 rows x 4 columns], 'y_test_two_bed':        dec_median  jun_median  mar_median  sep_median
54            NaN         NaN         NaN         NaN
55            NaN         NaN         NaN         NaN
56            NaN         NaN         NaN         NaN
57            NaN         NaN         NaN         NaN
58            NaN         NaN  

### Feature Selection

In [ ]:
def pca_feature_selection(X_train, X_val, X_test, variance_threshold=0.9):
    """
    The function applies PCA for dimensionality reduction by fitting on the
    training set and transforming both the training and test sets. It keeps 
    a select number of components based on the defined variance 
    threshold. It returns the reduced training and test sets.
    """

    # Standardise the data
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_val_scaled = scaler.transform(X_val)
    X_test_scaled = scaler.transform(X_test)

    # Initialise PCA, specifying the variance threshold
    pca_temp = PCA().fit(X_train_scaled)
    cumulative_variance = pca_temp.explained_variance_ratio_.cumsum()
    # Find the number of components to capture the specified variance
    n_components = next(i for i, total_variance in enumerate(cumulative_variance) if total_variance >= variance_threshold) + 1

    pca = PCA(n_components=n_components)

    # Fit PCA on the training set and transform both training and test sets
    X_train_pca = pca.fit_transform(X_train_scaled)
    X_val_pca = pca.transform(X_val_scaled)
    X_test_pca = pca.transform(X_test_scaled)

    print(f"Number of components selected: {n_components}")
    print(f"Explained variance by selected components: {sum(pca.explained_variance_ratio_):.2f}")

    return X_train_pca, X_val_pca, X_test_pca

# Perform feature selection with PCA on the X sets 
X_train_one_bed, X_val_one_bed, X_test_one_bed = pca_feature_selection(X_train_one_bed, X_val_one_bed, X_test_one_bed)
X_train_two_bed, X_val_two_bed, X_test_two_bed = pca_feature_selection(X_train_two_bed, X_val_two_bed, X_test_two_bed)
X_train_three_bed, X_val_three_bed, X_test_three_bed = pca_feature_selection(X_train_three_bed, X_val_three_bed, X_test_three_bed)
X_train_two_bed_house, X_val_two_bed_house, X_test_two_bed_house = pca_feature_selection(X_train_two_bed_house, X_val_two_bed_house, X_test_two_bed_house)
X_train_three_bed_house, X_val_three_bed_house, X_test_three_bed_house = pca_feature_selection(X_train_three_bed_house, X_val_three_bed_house, X_test_three_bed_house)
X_train_four_bed_house, X_val_four_bed_house, X_test_four_bed_house = pca_feature_selection(X_train_four_bed_house, X_val_four_bed_house, X_test_four_bed_house)
X_train_all_properties, X_val_all_properties, X_test_all_properties = pca_feature_selection(X_train_all_properties, X_val_all_properties, X_test_all_properties)

Number of components selected: 3
Explained variance by selected components: 0.91
Number of components selected: 4
Explained variance by selected components: 0.93
Number of components selected: 3
Explained variance by selected components: 0.90
Number of components selected: 4
Explained variance by selected components: 0.91
Number of components selected: 5
Explained variance by selected components: 0.93
Number of components selected: 4
Explained variance by selected components: 0.92
Number of components selected: 5
Explained variance by selected components: 0.93


### Train & Fit Model

In [ ]:
# Define a list of all training, validation, and test sets
ML_dfs = [
    (X_train_one_bed, X_val_one_bed, X_test_one_bed, y_train_one_bed, y_val_one_bed, y_test_one_bed),
    (X_train_two_bed, X_val_two_bed, X_test_two_bed, y_train_two_bed, y_val_two_bed, y_test_two_bed),
    (X_train_three_bed, X_val_three_bed, X_test_three_bed, y_train_three_bed, y_val_three_bed, y_test_three_bed),
    (X_train_two_bed_house, X_val_two_bed_house, X_test_two_bed_house, y_train_two_bed_house, y_val_two_bed_house, y_test_two_bed_house),
    (X_train_three_bed_house, X_val_three_bed_house, X_test_three_bed_house, y_train_three_bed_house, y_val_three_bed_house, y_test_three_bed_house),
    (X_train_four_bed_house, X_val_four_bed_house, X_test_four_bed_house, y_train_four_bed_house, y_val_four_bed_house, y_test_four_bed_house),
    (X_train_all_properties, X_val_all_properties, X_test_all_properties, y_train_all_properties, y_val_all_properties, y_test_all_properties),
]

Lasso Regression

In [ ]:
# Define hyperparameters for Lasso
alpha_range = np.logspace(-6, 2, 20) 
param_grid = {'alpha': alpha_range}

# Loop through each set, tune the model on the validation set, and predict on the test set
for i, (X_train, X_val, X_test, y_train, y_val, y_test) in enumerate(ML_dfs):
    # Initialise the Lasso model
    lasso = Lasso()
    
    # Set up the GridSearchCV to tune the 'alpha' hyperparameter
    grid_search = GridSearchCV(lasso, param_grid, scoring='neg_mean_squared_error', cv=5)
    
    # Train the model using the training set and validate on the validation set
    grid_search.fit(X_train, y_train)
    
    # Select the best model based on the validation set
    best_lasso = grid_search.best_estimator_
    
    # Evaluate on the validation set
    y_val_pred = best_lasso.predict(X_val)
    val_mse = mean_squared_error(y_val, y_val_pred)
    val_r2 = r2_score(y_val, y_val_pred)
    
    # Make predictions on the test set
    y_test_pred = best_lasso.predict(X_test)
    
    # Print the results
    print(f"Dataset {i+1}:")
    print(f"Best alpha: {best_lasso.alpha}")
    print(f"Validation MSE: {val_mse:.4f}, R^2: {val_r2:.4f}")
    print(f"Predictions for 2025-2027: {y_test_pred}\n")


Dataset 1:
Best alpha: 0.7847599703514607
Validation MSE: 8159.8610, R^2: -0.4790
Predictions for 2025-2027: [[311.59193501 309.86959251 308.39247898 310.64367186]
 [311.59193501 309.86959251 308.39247898 310.64367186]
 [311.59193501 309.86959251 308.39247898 310.64367186]
 ...
 [298.11052651 296.52401688 295.73514286 297.1122076 ]
 [298.11052651 296.52401688 295.73514286 297.1122076 ]
 [298.11052651 296.52401688 295.73514286 297.1122076 ]]

Dataset 2:
Best alpha: 14.384498882876601
Validation MSE: 13294.3083, R^2: -0.5591
Predictions for 2025-2027: [[387.48994133 384.54625171 383.0489088  386.14617014]
 [387.48994133 384.54625171 383.0489088  386.14617014]
 [387.48994133 384.54625171 383.0489088  386.14617014]
 ...
 [380.40630813 377.73638583 376.37806622 379.12463257]
 [380.40630813 377.73638583 376.37806622 379.12463257]
 [380.40630813 377.73638583 376.37806622 379.12463257]]

Dataset 3:
Best alpha: 100.0
Validation MSE: 38661.7270, R^2: -0.2995
Predictions for 2025-2027: [[510.0353

/Users/promaali/Desktop/Desktop Documents/Unimelb 2024/Sem 2/Applied Data Science/GitHub/project-2-group-real-estate-industry-project-15/.venv/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.263e+06, tolerance: 5.127e+03
  model = cd_fast.enet_coordinate_descent(
/Users/promaali/Desktop/Desktop Documents/Unimelb 2024/Sem 2/Applied Data Science/GitHub/project-2-group-real-estate-industry-project-15/.venv/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.168e+06, tolerance: 5.205e+03
  model = cd_fast.enet_coordinate_descent(
/Users/promaali/Desktop/Desktop Docume

Dataset 4:
Best alpha: 37.92690190732246
Validation MSE: 19088.4432, R^2: -0.3888
Predictions for 2025-2027: [[417.53059699 412.54599905 410.97161334 415.51109802]
 [417.53059699 412.54599905 410.97161334 415.51109802]
 [417.53059699 412.54599905 410.97161334 415.51109802]
 ...
 [411.98139506 407.4709226  405.49147045 409.92761949]
 [411.98139506 407.4709226  405.49147045 409.92761949]
 [411.98139506 407.4709226  405.49147045 409.92761949]]

Dataset 5:
Best alpha: 37.92690190732246
Validation MSE: 39802.8284, R^2: -0.2696
Predictions for 2025-2027: [[490.01924086 483.78389201 480.19659156 487.60805463]
 [490.01924086 483.78389201 480.19659156 487.60805463]
 [490.01924086 483.78389201 480.19659156 487.60805463]
 ...
 [491.40617474 486.33878061 482.99290193 489.27593669]
 [491.40617474 486.33878061 482.99290193 489.27593669]
 [491.40617474 486.33878061 482.99290193 489.27593669]]



/Users/promaali/Desktop/Desktop Documents/Unimelb 2024/Sem 2/Applied Data Science/GitHub/project-2-group-real-estate-industry-project-15/.venv/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.211e+08, tolerance: 2.479e+04
  model = cd_fast.enet_coordinate_descent(
/Users/promaali/Desktop/Desktop Documents/Unimelb 2024/Sem 2/Applied Data Science/GitHub/project-2-group-real-estate-industry-project-15/.venv/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.230e+08, tolerance: 2.522e+04
  model = cd_fast.enet_coordinate_descent(
/Users/promaali/Desktop/Desktop Docume

Dataset 6:
Best alpha: 14.384498882876601
Validation MSE: 84097.1932, R^2: -0.2328
Predictions for 2025-2027: [[654.41658134 651.4694208  647.72471858 654.03609315]
 [654.41658134 651.4694208  647.72471858 654.03609315]
 [654.41658134 651.4694208  647.72471858 654.03609315]
 ...
 [613.65922284 607.40515935 603.74444741 611.11293446]
 [613.65922284 607.40515935 603.74444741 611.11293446]
 [613.65922284 607.40515935 603.74444741 611.11293446]]

Dataset 7:
Best alpha: 100.0
Validation MSE: 15682.9773, R^2: -0.7938
Predictions for 2025-2027: [[408.57996633 404.12205387 402.64983165 406.42929293]
 [408.57996633 404.12205387 402.64983165 406.42929293]
 [408.57996633 404.12205387 402.64983165 406.42929293]
 ...
 [408.57996633 404.12205387 402.64983165 406.42929293]
 [408.57996633 404.12205387 402.64983165 406.42929293]
 [408.57996633 404.12205387 402.64983165 406.42929293]]



Random Forest

In [ ]:
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.model_selection import GridSearchCV
# from sklearn.metrics import mean_squared_error, r2_score
# import numpy as np

# # Define the hyperparameters for Random Forest
# param_grid = {
#     'n_estimators': [50, 100, 200],  # Number of trees
#     'max_depth': [5, 10, 15, None],  # Maximum depth of each tree
# }

# # Loop through each set, tune the model on the validation set, and predict on the test set
# for i, (X_train, X_val, X_test, y_train, y_val, y_test) in enumerate(ML_dfs):
#     # Initialise the Random Forest model
#     rf = RandomForestRegressor(random_state=42)
    
#     # Set up the GridSearchCV to tune 'n_estimators' and 'max_depth'
#     grid_search = GridSearchCV(rf, param_grid, scoring='neg_mean_squared_error', cv=5)
    
#     # Train the model using the training set and validate on the validation set
#     grid_search.fit(X_train, y_train)
    
#     # Select the best model based on the validation set
#     best_rf = grid_search.best_estimator_
    
#     # Evaluate on the validation set
#     y_val_pred = best_rf.predict(X_val)
#     val_mse = mean_squared_error(y_val, y_val_pred)
#     val_r2 = r2_score(y_val, y_val_pred)
    
#     # Make predictions on the test set
#     y_test_pred = best_rf.predict(X_test)
    
#     # Print the results
#     print(f"Dataset {i+1}:")
#     print(f"Best n_estimators: {best_rf.n_estimators}, Best max_depth: {best_rf.max_depth}")
#     print(f"Validation MSE: {val_mse:.4f}, R^2: {val_r2:.4f}")
#     print(f"Predictions for 2025-2027: {y_test_pred}\n")


Arima Modelling

In [ ]:
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error, r2_score

# Assuming 'y_train', 'y_val', and 'y_test' are DataFrames with multiple target columns
target_columns = ['dec_median', 'jun_median', 'mar_median', 'sep_median']  # Adjust according to your target columns

# Loop through each dataset
for i, (X_train, X_val, X_test, y_train, y_val, y_test) in enumerate(ML_dfs):
    print(f"Dataset {i+1} results:")
    
    # Apply ARIMA for each target variable
    for target in target_columns:
        print(f"Fitting ARIMA for target: {target}...")

        # Extract the target series for ARIMA
        y_train_series = y_train[target]
        y_val_series = y_val[target]
        y_test_series = y_test[target]
        
        # Define the ARIMA order (p, d, q)
        p = 5  # AR term
        d = 1  # Differencing term
        q = 0  # MA term
        
        # Fit ARIMA model to the training data
        model = ARIMA(y_train_series, order=(p, d, q))
        model_fit = model.fit()
        
        # Predict on the validation set
        y_val_pred = model_fit.forecast(steps=len(y_val_series))
        
        # Calculate validation MSE and R^2
        val_mse = mean_squared_error(y_val_series, y_val_pred)
        val_r2 = r2_score(y_val_series, y_val_pred)
        
        # Predict on the test set
        y_test_pred = model_fit.forecast(steps=len(y_test_series))
        
        # Print the results
        print(f"Target: {target}")
        print(f"Validation MSE: {val_mse:.4f}, Validation R^2: {val_r2:.4f}")
        print(f"Test predictions for 2025-2027: {y_test_pred}\n")


Dataset 1 results:
Fitting ARIMA for target: dec_median...
Target: dec_median
Validation MSE: 11924.0906, Validation R^2: -1.3932
Test predictions for 2025-2027: 4710    270.0
4711    270.0
4712    270.0
4713    270.0
4714    270.0
        ...  
7081    270.0
7082    270.0
7083    270.0
7084    270.0
7085    270.0
Name: predicted_mean, Length: 2376, dtype: float64

Fitting ARIMA for target: jun_median...


/Users/promaali/Desktop/Desktop Documents/Unimelb 2024/Sem 2/Applied Data Science/GitHub/project-2-group-real-estate-industry-project-15/.venv/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the supported classes of index.
  self._init_dates(dates, freq)
/Users/promaali/Desktop/Desktop Documents/Unimelb 2024/Sem 2/Applied Data Science/GitHub/project-2-group-real-estate-industry-project-15/.venv/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the supported classes of index.
  self._init_dates(dates, freq)
/Users/promaali/Desktop/Desktop Documents/Unimelb 2024/Sem 2/Applied Data Science/GitHub/project-2-group-real-estate-industry-project-15/.venv/lib/python3.12/site-packages/st

Target: jun_median
Validation MSE: 8514.9311, Validation R^2: -0.8206
Test predictions for 2025-2027: 4710    278.0
4711    278.0
4712    278.0
4713    278.0
4714    278.0
        ...  
7081    278.0
7082    278.0
7083    278.0
7084    278.0
7085    278.0
Name: predicted_mean, Length: 2376, dtype: float64

Fitting ARIMA for target: mar_median...


/Users/promaali/Desktop/Desktop Documents/Unimelb 2024/Sem 2/Applied Data Science/GitHub/project-2-group-real-estate-industry-project-15/.venv/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/Users/promaali/Desktop/Desktop Documents/Unimelb 2024/Sem 2/Applied Data Science/GitHub/project-2-group-real-estate-industry-project-15/.venv/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:837: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(
/Users/promaali/Desktop/Desktop Documents/Unimelb 2024/Sem 2/Applied Data Science/GitHub/project-2-group-real-estate-industry-project-15/.venv/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:837: ValueWarning: No supported

Target: mar_median
Validation MSE: 6585.3367, Validation R^2: -0.4327
Test predictions for 2025-2027: 4710    290.0
4711    290.0
4712    290.0
4713    290.0
4714    290.0
        ...  
7081    290.0
7082    290.0
7083    290.0
7084    290.0
7085    290.0
Name: predicted_mean, Length: 2376, dtype: float64

Fitting ARIMA for target: sep_median...
Target: sep_median
Validation MSE: 15270.8297, Validation R^2: -0.9631
Test predictions for 2025-2027: 4710    270.0
4711    270.0
4712    270.0
4713    270.0
4714    270.0
        ...  
7081    270.0
7082    270.0
7083    270.0
7084    270.0
7085    270.0
Name: predicted_mean, Length: 2376, dtype: float64

Dataset 2 results:
Fitting ARIMA for target: dec_median...


...

/Users/promaali/Desktop/Desktop Documents/Unimelb 2024/Sem 2/Applied Data Science/GitHub/project-2-group-real-estate-industry-project-15/.venv/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the supported classes of index.
  self._init_dates(dates, freq)
/Users/promaali/Desktop/Desktop Documents/Unimelb 2024/Sem 2/Applied Data Science/GitHub/project-2-group-real-estate-industry-project-15/.venv/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the supported classes of index.
  self._init_dates(dates, freq)
/Users/promaali/Desktop/Desktop Documents/Unimelb 2024/Sem 2/Applied Data Science/GitHub/project-2-group-real-estate-industry-project-15/.venv/lib/python3.12/site-packages/st

Target: jun_median
Validation MSE: 7293.6448, Validation R^2: -0.0013
Test predictions for 2025-2027: 7128     475.0
7129     475.0
7130     475.0
7131     475.0
7132     475.0
         ...  
10687    475.0
10688    475.0
10689    475.0
10690    475.0
10691    475.0
Name: predicted_mean, Length: 3564, dtype: float64

Fitting ARIMA for target: mar_median...
Target: mar_median
Validation MSE: 7521.1616, Validation R^2: -0.0107
Test predictions for 2025-2027: 7128     480.0
7129     480.0
7130     480.0
7131     480.0
7132     480.0
         ...  
10687    480.0
10688    480.0
10689    480.0
10690    480.0
10691    480.0
Name: predicted_mean, Length: 3564, dtype: float64

Fitting ARIMA for target: sep_median...


/Users/promaali/Desktop/Desktop Documents/Unimelb 2024/Sem 2/Applied Data Science/GitHub/project-2-group-real-estate-industry-project-15/.venv/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/Users/promaali/Desktop/Desktop Documents/Unimelb 2024/Sem 2/Applied Data Science/GitHub/project-2-group-real-estate-industry-project-15/.venv/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:837: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(
/Users/promaali/Desktop/Desktop Documents/Unimelb 2024/Sem 2/Applied Data Science/GitHub/project-2-group-real-estate-industry-project-15/.venv/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:837: ValueWarning: No supported

Target: sep_median
Validation MSE: 13800.4104, Validation R^2: -0.0584
Test predictions for 2025-2027: 7128     480.0
7129     480.0
7130     480.0
7131     480.0
7132     480.0
         ...  
10687    480.0
10688    480.0
10689    480.0
10690    480.0
10691    480.0
Name: predicted_mean, Length: 3564, dtype: float64



/Users/promaali/Desktop/Desktop Documents/Unimelb 2024/Sem 2/Applied Data Science/GitHub/project-2-group-real-estate-industry-project-15/.venv/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/Users/promaali/Desktop/Desktop Documents/Unimelb 2024/Sem 2/Applied Data Science/GitHub/project-2-group-real-estate-industry-project-15/.venv/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:837: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(
